In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import psycopg2
from datetime import datetime, timedelta

In [3]:
try:
    cpo = psycopg2.connect(host="35.240.137.10",
                            database="prd_order",
                            user="akhiyar_waladi",
                            password="nd4n6fk9")
    print("prd_order Connected!")
except Exception as error:
    print("Error while connecting: ", error)
    sys.exit()

prd_order Connected!


In [4]:
try:
    cpp = psycopg2.connect(host="35.240.166.149",
                            database="prd_promo_stl_v2",
                            user="akhiyar_waladi",
                            password="nd4n6fk9")
    print("prd_promo Connected!")
except Exception as error:
    print("Error while connecting: ", error)
    sys.exit()

prd_promo Connected!


In [6]:
## loop from start that we want
for start_date in pd.date_range('2021-06-12', '2021-06-14', freq='D'):
    
    end_date = start_date + timedelta(days=1)
    
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    print(start_date_str, end_date_str)

    q_order = '''
    select tbto_id, tbto_no, tbtc_id, tbto_voucher_code, tbto_create_date, tbto_status
    from tb_transaction_order tto
    where tto.tbto_create_date between '{}' and '{}'
    and tbto_voucher_code != ''
    and tbto_status in ('12','14','15')

    '''.format(start_date_str, end_date_str)
    df_order = pd.read_sql(q_order, cpo)

    q_promo = '''
    select tbtvi_cart_id, tbtvi_registration_receipt, tbtvi_registration_no, tbtvi_created_date, tbtvi_updated_date, tbtvi_released 
    from tb_transaction_voucher_issued ttvi 
    where tbtvi_cart_id in {}
    '''.format(tuple(df_order['tbtc_id']))
    df_promo = pd.read_sql(q_promo, cpp)

    df_order_promo = pd.merge(df_order, df_promo, left_on='tbtc_id', right_on='tbtvi_cart_id')

    df_order_promo[df_order_promo['tbtvi_released'] == True].\
        to_excel('/home/server/gli-data-science/akhiyar/used_release/used_in_release_{}.xlsx'.format(start_date), index=False)

2021-06-12 2021-06-13
2021-06-13 2021-06-14
2021-06-14 2021-06-15


In [ ]:
### for testing purpose uncomment this ###

# start_date_str = '2021-04-05'
# end_date_str = '2021-04-06'
# q_order = '''
# select tbto_id, tbto_no, tbtc_id, tbto_voucher_code, tbto_create_date, tbto_status
# from tb_transaction_order tto
# where tto.tbto_create_date between '{}' and '{}'
# and tbto_voucher_code != ''
# and tbto_status in ('12','14','15')

# '''.format(start_date_str, end_date_str)
# df_order = pd.read_sql(q_order, cpo)

# q_promo = '''
# select tbtvi_cart_id, tbtvi_registration_receipt, tbtvi_registration_no, tbtvi_created_date, tbtvi_updated_date, tbtvi_released 
# from tb_transaction_voucher_issued ttvi 
# where tbtvi_cart_id in {}
# '''.format(tuple(list(df_order['tbtc_id'])))
# df_promo = pd.read_sql(q_promo, cpp)

# df_join =  pd.merge(df_order, df_promo, left_on='tbtc_id', right_on='tbtvi_cart_id', how='left')

# df_join.to_excel('reiBangke_{}_{}.xlsx'.format(start_date_str, end_date_str), index=False)